# Infrastructure_and_Models
**Purpose:** Deterministic environment setup, global configuration, and base model provenance.

**Scope:**
* Sets up the PyTorch environment and global seeds.
* Defines the central configuration (paths, precision, hyperparameters).
* Loads and freezes the base Stable Diffusion backbone (UNet, VAE, Text Encoder).
* Performs integrity checks on loaded weights.

**Exclusions:**
* No image generation or sampling.
* No training or optimization loops.
* No visualization.

## Reproducibility Contract
* **Deterministic Seeds:** `42` is used as the global seed for Python, NumPy, and PyTorch.
* **Hardware Config:** Precision is locked to `float16` (or `bfloat16` if available) on CUDA.
* **Artifacts:** All models loaded here form the immutable basis for subsequent experiments.
* **Persistence:** Configuration and model metadata are saved to `outputs/init_snapshot.json`.

In [ ]:
import sys
import torch
import diffusers
import transformers
import numpy as np

print(f"Python Version: {sys.version.split()[0]}")
print(f"PyTorch Version: {torch.__version__}")
print(f"Diffusers Version: {diffusers.__version__}")
print(f"Transformers Version: {transformers.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")

In [ ]:
import os
import random
import json
import warnings
import logging
from pathlib import Path

import torch
import torch.nn as nn
import numpy as np
from diffusers import StableDiffusionPipeline, DDIMScheduler
from transformers import CLIPTextModel, CLIPTokenizer, CLIPImageProcessor

warnings.filterwarnings("ignore")
logging.getLogger("diffusers").setLevel(logging.ERROR)
logging.getLogger("transformers").setLevel(logging.ERROR)

In [ ]:
def seed_everything(seed=42):
    """
    Enforces deterministic seeding across all libraries.
    """
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"Global seed set to: {seed}")

GLOBAL_SEED = 42
seed_everything(GLOBAL_SEED)

In [ ]:
class Config:
    """
    Central configuration for the entire research pipeline.
    """
    PROJECT_NAME = "Latent_Prompt_Injection"
    Seed = GLOBAL_SEED
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    PRECISION = torch.float16 
    MODEL_ID = "runwayml/stable-diffusion-v1-5"
    SCHEDULER_ID = "runwayml/stable-diffusion-v1-5"
    OUTPUT_ROOT = Path("experiments")
    
    LATENT_DIM = 4
    LATENT_RES = 64
    MAX_SEQ_LEN = 77

config = Config()
print(f"Configuration loaded. Device: {config.DEVICE}, Precision: {config.PRECISION}")

In [ ]:
def setup_directories(root_dir):
    """
    Creates the standard directory tree for experiment artifacts.
    """
    dirs = {
        "outputs": root_dir / "outputs",
        "checkpoints": root_dir / "checkpoints",
        "logs": root_dir / "logs",
        "figures": root_dir / "figures",
        "metadata": root_dir / "metadata"
    }
    
    for key, path in dirs.items():
        path.mkdir(parents=True, exist_ok=True)
        print(f"Confirmed directory: {path}")
        
    return dirs

dirs = setup_directories(config.OUTPUT_ROOT)

In [ ]:
def load_base_models(cfg):
    """
    Loads and freezes the base Stable Diffusion components.
    Returns individual components for granular control in N2.
    """
    print(f"Loading models from {cfg.MODEL_ID}...")
    scheduler = DDIMScheduler.from_pretrained(cfg.MODEL_ID, subfolder="scheduler")
    
    vae = StableDiffusionPipeline.from_pretrained(
        cfg.MODEL_ID, 
        subfolder="vae",
        torch_dtype=cfg.PRECISION
    ).vae.to(cfg.DEVICE)
    
    unet = StableDiffusionPipeline.from_pretrained(
        cfg.MODEL_ID, 
        subfolder="unet",
        torch_dtype=cfg.PRECISION
    ).unet.to(cfg.DEVICE)

    text_encoder = CLIPTextModel.from_pretrained(
        cfg.MODEL_ID, 
        subfolder="text_encoder", 
        torch_dtype=cfg.PRECISION
    ).to(cfg.DEVICE)
    tokenizer = CLIPTokenizer.from_pretrained(cfg.MODEL_ID, subfolder="tokenizer")
    
    vae.requires_grad_(False)
    unet.requires_grad_(False)
    text_encoder.requires_grad_(False)
    return tokenizer, text_encoder, unet, vae, scheduler

tokenizer, text_encoder, unet, vae, scheduler = load_base_models(config)

In [ ]:
print("Performing Integrity checks")

assert text_encoder.device.type == config.DEVICE.type, "Text Encoder on wrong device"
assert unet.device.type == config.DEVICE.type, "UNet on wrong device"
assert vae.device.type == config.DEVICE.type, "VAE on wrong device"
assert unet.dtype == config.PRECISION, f"UNet dtype mismatch. Expected {config.PRECISION}, got {unet.dtype}"

unet_params = sum(p.numel() for p in unet.parameters())
vae_params = sum(p.numel() for p in vae.parameters())
print(f"Integrity Verified.")
print(f"UNet Parameters:{unet_params:,}")
print(f"VAE Parameters:{vae_params:,}")

In [ ]:
model_registry = {
    "tokenizer": tokenizer,
    "text_encoder": text_encoder,
    "unet": unet,
    "vae": vae,
    "scheduler": scheduler,
    "config": config,
    "dirs": dirs
}

print("Model Registry initialized.")

## Setup Complete
**Artifacts Produced:**
1.  Locked Global Seed (`42`)
2.  Frozen Model Registry (UNet, VAE, Text Encoder)
3.  Experiment Directory Structure
4.  `init_snapshot.json` for audit

**Next Steps:**
These artifacts are now ready to be consumed by **Notebook 02: `02_latent_manipulation_and_generation.ipynb`** for the core latent prompt injection methodology.